Dieses Programm stellt ein FEM einen graden Stabes dar, der entlag seiner Ausdehnung belastet wird.
Der Stab ist dabei in eine drei Teilstäbe unterteilt, die jeweils ihre eigenen Eigenschaften haben.

Für die Verarbeitung der Matrizien nutze ich numpy. Numpy ist eine Python Bibliothek die zum großen Teil in C geschrieben ist und somit die Rechnenleistung effiziernter nutzt. 
Bitte den Block zur installertion des Paketes nur ausführen, wenn noch keine Installation vorliegt. 

In [1]:
# pip install numpy


In [2]:
import numpy as np

Im Folgenden definiere ich die Parameter für die Aufgabenstellung. 

L_n  < >  Längen der Teilstäbe.

A    < >  Querschnittsfläche der Teilstäbe.

E    < >  Elastitizitätsmodul der Teilstäbe.

n_e  < >  Anzahl der Teilstäbe. 

In [3]:
L_n = [L_1, L_2, L_3] = [0.4, 0.3, 0.2]     
A_n = [A_1, A_2, A_3] = [2.0, 1.0, 0.5]
E_n = [E_1, E_2, E_3] = [200.0, 100.0, 50.0]
n_e = 3

Zwei weitere Parameter ergeben sich direkt aus den gegebenen Größen.

n_k < > Anzahl der Knoten

L_sum < > Die Gesamtlänge des Stabes

k_n < > Koordinaten der Knoten


In [7]:
n_k = n_e + 1
L_sum = sum(L_n)
k_n = []
for i in range(n_e):
    k_n.append(sum(L_n[0:i]))

Nun stellen wir die Steifigkeitsmatrix auf.  
$$ 
K  = \frac{A \cdot E}{ \frac{L}{n_e}} \begin{bmatrix}
1 & -1 \\
-1 & 1
\end{bmatrix}

$$

In [13]:
K_local = []
for L, A, E in zip(L_n, A_n, E_n):
    K_l = (A*E/L)* np.array([[1, -1], [-1, 1]])
    print(K_l)
    K_local.append(K_l)
print(K_local)

[[ 1000. -1000.]
 [-1000.  1000.]]
[[ 333.33333333 -333.33333333]
 [-333.33333333  333.33333333]]
[[ 125. -125.]
 [-125.  125.]]
[array([[ 1000., -1000.],
       [-1000.,  1000.]]), array([[ 333.33333333, -333.33333333],
       [-333.33333333,  333.33333333]]), array([[ 125., -125.],
       [-125.,  125.]])]


Die globale Steifigkeitsmatrix setzt sich aus den Steifigekeitsmatrizen der Knoten zusammen. 
Der ausdruck i:i+2 und i:i+2 ist daafür verantwortlich, dass sich die jeweiligen Matrizen auf einer Diagonalen in der allgeimnen Matrix wiederfinden. 

In [17]:
K = np.zeros((n_k, n_k))
for i, K_l in enumerate(K_local):
    K[i:i+2, i:i+2] += K_l
print(K)

[[ 1000.         -1000.             0.             0.        ]
 [-1000.          1333.33333333  -333.33333333     0.        ]
 [    0.          -333.33333333   458.33333333  -125.        ]
 [    0.             0.          -125.           125.        ]]


Nun gestalte ich einen Lastvektor.np.zero gibt dabei einen Voktor voller Nullen zurück. 


In [18]:
F = np.zeros(n_k)
F[-1] = 10.0

Nun legen wir die Anfangsbedingungen fest. 

In [19]:
K[0,:] = 0  # Erste Zeile wird gleich null gesetzt
K[:,0] = 0  # Erste Spalte wird gleich null gesetzt
K[0,0] = 1  # Erstes Element wird gleich eins gesetzt, somit bleibt matrix invertiertbar
F[0] = 0    # Keine Kraft wirkt am ersten Knoten

Bevor ich das Gleichungssystem löse, lasse ich die Matrix und den Vektor ausgeben.

In [20]:
print("Globale Steifigkeitsmatrix K:")
print(K)
print("\nLastvektor F:")
print(F)

Globale Steifigkeitsmatrix K:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.33333333e+03 -3.33333333e+02  0.00000000e+00]
 [ 0.00000000e+00 -3.33333333e+02  4.58333333e+02 -1.25000000e+02]
 [ 0.00000000e+00  0.00000000e+00 -1.25000000e+02  1.25000000e+02]]

Lastvektor F:
[ 0.  0.  0. 10.]


Nun Lösen wir das Gleichungssystem und geben die Lösung aus.    

In [21]:
dis  = np.linalg.solve(K, F)
print(dis)

[0.   0.01 0.04 0.12]
